In [1]:
import pandas as pd
import os
import re
import numpy as np
from datetime import datetime, timedelta
import time
import pickle

In [61]:
os.chdir('C:\\DynamicData')

product = pd.read_csv('product.csv', encoding='utf-8')
session = pd.read_csv('Session.csv', encoding='utf-8')

C:\Users\anaconda\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### product 전처리

In [5]:
product['PD_BUY_AM'] = list(map(lambda x:re.sub("\,", "",x), product['PD_BUY_AM']))
product['PD_BUY_AM'] = product['PD_BUY_AM'].astype('int64')

In [7]:
product['PD_BUY_CT'] = product['PD_BUY_CT'].astype('str')
product['PD_BUY_CT'] = list(map(lambda x:re.sub("\,", "",x), product['PD_BUY_CT']))
product['PD_BUY_CT'] = product['PD_BUY_CT'].astype('int64')

### session 데이터

### Session에 휴일여부 추가하기

In [3]:
session['Timestamp'] = 0

In [4]:
session['SESS_DT'] = session['SESS_DT'].astype('str')
session['SESS_DT'] = list(map(lambda x:datetime.strptime(x,'%Y%m%d'), session['SESS_DT']))
session['Timestamp'] = list(map(lambda x:datetime.timestamp(x), session['SESS_DT']))

In [5]:
Sat = pd.date_range(min(session.SESS_DT), max(session.SESS_DT), freq='W-SAT') #Saturday
Sun = pd.date_range(min(session.SESS_DT), max(session.SESS_DT), freq='W-SUN') #Sunday

In [6]:
Sat_Timestamp=list(map(lambda x:datetime.timestamp(x), Sat))
Sun_Timestamp=list(map(lambda x:datetime.timestamp(x), Sun))

In [7]:
Holiday = np.array(['20180505', '20180522', '20180606', '20180815', '20180923', '20180924', '20180925', ])
repic_Holiday = np.array(['20180507', '20180613', '20180926']) #5월7일, 9월26일: 대체 공휴일,6월 13일: 지방선거일
Holiday = np.append(Holiday, repic_Holiday)
Holiday = list(map(lambda x:datetime.strptime(x,'%Y%m%d'), Holiday))

In [8]:
Holiday_Timestamp=list(map(lambda x:datetime.timestamp(x), Holiday))

In [9]:
All_Timestamp = Sat_Timestamp + Sun_Timestamp + Holiday_Timestamp

In [10]:
session['Rest'] = 0 #휴일인 날

In [11]:
session.ix[session['Timestamp'].isin(All_Timestamp),'Rest']=1

C:\Users\anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [21]:
weekend_Timestamp = Sat_Timestamp + Sun_Timestamp
weekend_Timestamp = list(map(lambda x:datetime.fromtimestamp(x), weekend_Timestamp))

session['five_before'] = 0
session.ix[session['SESS_DT'].isin(list(map(lambda x: x+timedelta(days=-5), weekend_Timestamp))), 'five_before'] = 1

C:\Users\anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


In [22]:
session

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,Timestamp,Rest,five_before
0,5873599,8641867,9,2018-05-09,82.0,"1,890",mobile,Daejeon,Daejeon,1.525792e+09,0,0
1,5873599,6616320,21,2018-06-11,105.0,"1,604",mobile,Busan,Busan,1.528643e+09,0,1
2,5873599,5886172,40,2018-06-24,41.0,632,mobile,Daejeon,Daejeon,1.529766e+09,1,0
3,5873884,1050889,15,2018-09-13,160.0,"1,035",mobile,Gyeonggi-do,Anyang,1.536764e+09,0,0
4,5874461,10298270,5,2018-04-12,13.0,298,mobile,Seoul,Seoul,1.523459e+09,0,0
5,5874701,608577,65,2018-09-20,41.0,444,mobile,Seoul,Seoul,1.537369e+09,0,0
6,5874829,1666766,87,2018-09-03,36.0,340,mobile,Seoul,Seoul,1.535900e+09,0,1
7,5874829,1670494,86,2018-09-03,103.0,976,mobile,Seoul,Seoul,1.535900e+09,0,1
8,5874829,1694242,85,2018-09-02,154.0,"3,099",mobile,Seoul,Seoul,1.535814e+09,1,0
9,5874829,1559438,92,2018-09-04,24.0,317,mobile,Seoul,Seoul,1.535987e+09,0,1


In [51]:
children_day = np.array(['2018-05-05'])
children_day = list(map(lambda x:datetime.strptime(x,'%Y-%m-%d'), children_day))
children_hotdays = []

children_hotdays.append(list(map(lambda x: x+timedelta(days=-3), children_day))[0])
children_hotdays.append(list(map(lambda x: x+timedelta(days=-4), children_day))[0])
children_hotdays.append(list(map(lambda x: x+timedelta(days=-5), children_day))[0])
children_hotdays.append(list(map(lambda x: x+timedelta(days=-6), children_day))[0])

session['children_hotday'] = 0
session.ix[session['SESS_DT'].isin(children_hotdays),'children_hotday'] = 1

C:\Users\anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()


In [52]:
teacher_day = np.array(['2018-05-15'])
teacher_day = list(map(lambda x:datetime.strptime(x,'%Y-%m-%d'), teacher_day))
teacher_hotdays = []

teacher_hotdays.append(list(map(lambda x: x+timedelta(days=-1), teacher_day))[0])

session['teacher_hotday'] = 0
session.ix[session['SESS_DT'].isin(teacher_hotdays), 'teacher_hotday'] = 1

C:\Users\anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [54]:
election_day = np.array(['2018-06-13']) 
election_day = list(map(lambda x:datetime.strptime(x,'%Y-%m-%d'), election_day))

election_hotdays = []

election_hotdays.append(list(map(lambda x: x+timedelta(days=-2), election_day))[0])
election_hotdays.append(list(map(lambda x: x+timedelta(days=-3), election_day))[0])

session['election_hotday'] = 0
session.ix[session['SESS_DT'].isin(election_hotdays), 'election_hotday'] = 1

C:\Users\anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.


In [60]:
with open('Preprosessing_Robin', 'wb') as wb:
    pickle.dump(session, wb)